In [1]:
# Imports
import numpy as np

In [3]:
# Loading training data
data = open('kafka.txt', 'r').read()

chars = list(set(data)) 
data_size, vocab_size = len(data), len(chars)
print('data has %d chars, %d unique' % (data_size, vocab_size))

data has 137628 chars, 80 unique


In [5]:
# Creating 2 dicts to encode and decode char to int
char_to_ix = { ch:i for i,ch in enumerate(chars)}
ix_to_char = { i:ch for i, ch in enumerate(chars)}
print(char_to_ix)
print(ix_to_char)

{'r': 0, ' ': 1, 'E': 2, 'f': 3, 'v': 4, '-': 5, '!': 6, ')': 7, '$': 8, 'K': 9, 'm': 10, 'O': 11, 'o': 12, 'g': 13, 'd': 14, '?': 15, '%': 16, 'M': 17, 'R': 18, ':': 19, 'Q': 20, 'w': 21, 'L': 22, 'C': 23, 'F': 24, 'V': 25, "'": 26, 'U': 27, '1': 28, 'D': 29, 'ç': 30, 'T': 31, 'j': 32, ';': 33, 'k': 34, 'a': 35, 'S': 36, 'z': 37, '"': 38, '5': 39, 'X': 40, 'h': 41, 'l': 42, 'J': 43, '/': 44, '3': 45, '4': 46, '@': 47, 'N': 48, '9': 49, 'e': 50, 'y': 51, 'Y': 52, 'A': 53, '*': 54, ',': 55, '.': 56, 'q': 57, '\n': 58, 'I': 59, '2': 60, '0': 61, 'p': 62, 'c': 63, 'x': 64, '6': 65, 't': 66, 'n': 67, 'u': 68, 'B': 69, '8': 70, '7': 71, 'b': 72, 's': 73, 'H': 74, 'W': 75, 'P': 76, 'i': 77, '(': 78, 'G': 79}
{0: 'r', 1: ' ', 2: 'E', 3: 'f', 4: 'v', 5: '-', 6: '!', 7: ')', 8: '$', 9: 'K', 10: 'm', 11: 'O', 12: 'o', 13: 'g', 14: 'd', 15: '?', 16: '%', 17: 'M', 18: 'R', 19: ':', 20: 'Q', 21: 'w', 22: 'L', 23: 'C', 24: 'F', 25: 'V', 26: "'", 27: 'U', 28: '1', 29: 'D', 30: 'ç', 31: 'T', 32: 'j', 

In [6]:
# Creating char vectors
vector_for_char_a = np.zeros((vocab_size, 1))
vector_for_char_a[char_to_ix['a']] = 1
print(vector_for_char_a.ravel())

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]


In [7]:
# hyperparameters of model
hidden_size = 100
seq_length = 25
learning_rate = 1e-1

In [8]:
# model parameters
Wxh = np.random.randn(hidden_size, vocab_size) * 0.01 #input to hidden
Whh = np.random.randn(hidden_size, hidden_size) * 0.01 #hidden to hidden
Why = np.random.randn(vocab_size, hidden_size) * 0.01 #hidden to output
bh = np.zeros((hidden_size, 1))
by = np.zeros((vocab_size, 1))

In [16]:
def lossFun(inputs, targets, hprev):
  """                                                                                                                                                                                         
  inputs,targets are both list of integers.                                                                                                                                                   
  hprev is Hx1 array of initial hidden state                                                                                                                                                  
  returns the loss, gradients on model parameters, and last hidden state                                                                                                                      
  """
  #store our inputs, hidden states, outputs, and probability values
  xs, hs, ys, ps, = {}, {}, {}, {} #Empty dicts
    # Each of these are going to be SEQ_LENGTH(Here 25) long dicts i.e. 1 vector per time(seq) step
    # xs will store 1 hot encoded input characters for each of 25 time steps (26, 25 times)
    # hs will store hidden state outputs for 25 time steps (100, 25 times)) plus a -1 indexed initial state
    # to calculate the hidden state at t = 0
    # ys will store targets i.e. expected outputs for 25 times (26, 25 times), unnormalized probabs
    # ps will take the ys and convert them to normalized probab for chars
    # We could have used lists BUT we need an entry with -1 to calc the 0th hidden layer
    # -1 as  a list index would wrap around to the final element
  #init with previous hidden state
  # Using "=" would create a reference, this creates a whole separate copy
  # We don't want hs[-1] to automatically change if hprev is changed
  hs[-1] = np.copy(hprev)
  #init loss as 0
  loss = 0
  # forward pass                                                                                                                                                                              
  for t in range(len(inputs)):
    xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation (we place a 0 vector as the t-th input)                                                                                                                     
    xs[t][inputs[t]] = 1 # Inside that t-th input we use the integer in "inputs" list to  set the correct
    hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state                                                                                                            
    ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars                                                                                                           
    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars                                                                                                              
    loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)                                                                                                                       
  # backward pass: compute gradients going backwards    
  #initalize vectors for gradient values for each set of weights 
  dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
  dbh, dby = np.zeros_like(bh), np.zeros_like(by)
  dhnext = np.zeros_like(hs[0])
  for t in reversed(range(len(inputs))):
    #output probabilities
    dy = np.copy(ps[t])
    #derive our first gradient
    dy[targets[t]] -= 1 # backprop into y  
    #compute output gradient -  output times hidden states transpose
    #When we apply the transpose weight matrix,  
    #we can think intuitively of this as moving the error backward
    #through the network, giving us some sort of measure of the error 
    #at the output of the lth layer. 
    #output gradient
    dWhy += np.dot(dy, hs[t].T)
    #derivative of output bias
    dby += dy
    #backpropagate!
    dh = np.dot(Why.T, dy) + dhnext # backprop into h                                                                                                                                         
    dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity                                                                                                                     
    dbh += dhraw #derivative of hidden bias
    dWxh += np.dot(dhraw, xs[t].T) #derivative of input to hidden layer weight
    dWhh += np.dot(dhraw, hs[t-1].T) #derivative of hidden layer to hidden layer weight
    dhnext = np.dot(Whh.T, dhraw) 
  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients                                                                                                                 
  return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

In [17]:
#prediction, one full forward pass
def forward_pass(h, seed_ix, n):
  """                                                                                                                                                                                         
  sample a sequence of integers from the model                                                                                                                                                
  h is memory state, seed_ix is seed letter for first time step   
  n is how many characters to predict
  """
  #create vector
  x = np.zeros((vocab_size, 1))
  #customize it for our seed char
  x[seed_ix] = 1
  #list to store generated chars
  ixes = []
  #for as many characters as we want to generate
  for t in range(n):
    #a hidden state at a given time step is a function 
    #of the input at the same time step modified by a weight matrix 
    #added to the hidden state of the previous time step 
    #multiplied by its own hidden state to hidden state matrix.
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    #compute output (unnormalised)
    y = np.dot(Why, h) + by
    ## probabilities for next chars
    p = np.exp(y) / np.sum(np.exp(y))
    #pick one with the highest probability 
    ix = np.random.choice(range(vocab_size), p=p.ravel())
    #create a vector
    x = np.zeros((vocab_size, 1))
    #customize it for the predicted char
    x[ix] = 1
    #add it to the list
    ixes.append(ix)

  txt = ''.join(ix_to_char[ix] for ix in ixes)
  print('----\n %s \n----' % (txt, ))
hprev = np.zeros((hidden_size,1)) # reset RNN memory  
#predict the 200 next characters given 'a'
forward_pass(hprev,char_to_ix['a'],200)

----
 3MS$gUy%NdO,f
RLQ(i:FM"u20t-%lnb@(cT3ULl2
gjAz6k6HY7"$83$aS8B-kc8
qm2ATFpz3@jRsn-çIlq9-0lV"-dvx6R%"@wcUKG$aTIGUb:Jw -kwzw4*?i!J/Ia*)Jp"I/G?reJ!ç/:Px;QryPW$:%wKS9T2m.gNeYQe9U(qk-Mg zçtç*LDwTS'(uzctulf$ 
----


In [18]:
# Feeding loss function with inputs and targets
p=0  
inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
print("inputs", inputs)
targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]
print("targets", targets)

inputs [11, 67, 50, 1, 10, 12, 0, 67, 77, 67, 13, 55, 1, 21, 41, 50, 67, 1, 79, 0, 50, 13, 12, 0, 1]
targets [67, 50, 1, 10, 12, 0, 67, 77, 67, 13, 55, 1, 21, 41, 50, 67, 1, 79, 0, 50, 13, 12, 0, 1, 36]


In [20]:
n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad                                                                                                                
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0                                                                                                                        
while n<=1000*100:
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
  # check "How to feed the loss function to see how this part works
  if p+seq_length+1 >= len(data) or n == 0:
    hprev = np.zeros((hidden_size,1)) # reset RNN memory                                                                                                                                      
    p = 0 # go from start of data                                                                                                                                                             
  inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
  targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

  # forward seq_length characters through the net and fetch gradient                                                                                                                          
  loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
  smooth_loss = smooth_loss * 0.999 + loss * 0.001

  # sample from the model now and then                                                                                                                                                        
  if n % 1000 == 0:
    print('iter %d, loss: %f' % (n, smooth_loss)) # print progress
    forward_pass(hprev, inputs[0], 200)

  # perform parameter update with Adagrad                                                                                                                                                     
  for param, dparam, mem in zip([Wxh, Whh, Why, bh, by],
                                [dWxh, dWhh, dWhy, dbh, dby],
                                [mWxh, mWhh, mWhy, mbh, mby]):
    mem += dparam * dparam
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update                                                                                                                   

  p += seq_length # move data pointer                                                                                                                                                         
  n += 1 # iteration counter

iter 0, loss: 109.550658
----
 s%L)0cGç)oM'o;qY54/dFp9xa6b1ax;zsiS3xm"''%M$/V d.02
pm3t@*W
xh,)o:?K'yNs)qf-XxJl1YP?/;zpnXAz-KBFpI$8hqqch09L)N7lrEy8$@c4çç35VN% z8u6re05d*.zCIe9sg"oJFfoDNNJIud';Q@ulV@NqTGh$bbi.XDTA8JJieafl83)L0?$/qs@ 
----
iter 1000, loss: 82.593661
----
 ias ;ee chichit. -weiut trelf be ce hiof ve w" we "ist cann Thd Anf desuss se pe lis bef ey ceekoen cono il Gnd 
nasd ib thisf yistk ifss iule thace canghen hicd ant lot baplsildware freg this all "ar 
----
iter 2000, loss: 65.058914
----
 ty. ghenof and whinjeritor wo meavet ane. of iow wosless oouh. her eveleruinshamo mithen a?t mosedon nermle krfore- suaxkes aHy wire was tither wore hat eo seny has sisuliomimouid nra hisperesurheund  
----
iter 3000, loss: 56.152952
----
 t rasa hisled; ly the fedele tieven, that; her; ttours napmed fur at trre wind iistersurywate ane ansesimor his dither, beas buxtharsaading, wead out; the wooo enedestounte win to wermorolag ans shere 
----
iter 4000, loss: 51.974980
----
  The rior

KeyboardInterrupt: 